In [1]:
#Requires Python 3.8+ for inverse mod using pow()

# Imports

In [2]:
import random
import secrets
from decimal import *

# Functions

Returns the GCD of 2 numbers

In [3]:
def gcd(a, b):
    while(b):
        a, b = b, a % b 
    return a

Rabin-Miller Primality Test<br>
Checks if a number is prime, p is the number to be tested, s is the number of rounds<br>
Number of rounds should ensure that the probability of p not being prime is < 2^(-80) 

In [6]:
def RMPT(p, s):
    q = p - 1
    k = 0
    while q % 2 != 1:
        q //= 2
        k += 1
        
    for _ in range(s):
        a = random.randrange(3, p - 2)      
        while gcd(a, p) != 1:
            a = random.randrange(3, p - 2)
        
        z = pow(a, q, p)
        
        if z == 1 or z == p - 1:
            continue #Not a witness, pick another a
        
        for i in range(1, k):
            z = pow(a, pow(2, i)*q, p)
            if z == p - 1:
                break #Not a witness, pick another a
        else:
            return False  #a^((2^K)*Q)modP != -1 for all values of K, it is a witness, therefore its definitely not prime
        
    return True #After S iterations still not a composite, chance to be a prime

Converts a string to ascii

In [7]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

Converts ascii back to string

In [8]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

Generates the decryption key using public key e and 2 prime numbers p and q

In [9]:
def RSA_generate_D(e, p, q):
    N = p * q
    phi_N = (p - 1) * (q - 1)
    assert gcd(e, phi_N) == 1, "GCD of e and phi N must be 1"
    d = pow(e, -1, phi_N)
    return d

Encrypts a number with public key e

In [10]:
def RSA_encrypt(ascii_message, e, N):
    encrypted_message = pow(ascii_message, e, N)
    return encrypted_message

Encrypts a number using private key d

In [11]:
def RSA_decrypt(encrypted_message, d, N):
    ascii_message = pow(encrypted_message, d, N)
    return ascii_message

Continuously generates random bit_length numbers<br>
If number is even, adds one to make it odd<br>
Tries to factorize the number with small primes to determine if composite<br>
Uses Rabin-Miller Primality Test if it is still possible to be a prime number

In [12]:
def Generate_Prime(bit_length, RMPT_rounds):   
    while(True):
        num = secrets.randbits(bit_length)
        
        #If number is even, make it odd
        if num%2 == 0:
            num += 1
        #Try factorize using small primes
        elif num%3 == 0:
            continue
        elif num%5 == 0:
            continue
        elif num%7 == 0:
            continue
        elif num%11 == 0:
            continue
        #Try the expensive Rabin-Miller Primality Test
        elif RMPT(num, RMPT_rounds):
            print(num)
            return num

# Using RSA 1024 for signatures

Generate 2 random 1024 bit primes

In [289]:
p = Generate_Prime(1024, 3)

100090347480003983541165579353209902124921372611110263287717435567159178529917834236491103134949193993555895846827386848838450651095767610140329668234050707721046083642975420017210986781615040893826023713022996167967145675960726290036869387197959970805055944392314851818453665701264547981192967102582994573833


In [294]:
q = Generate_Prime(1024, 3)

91849429487171413566263377164551272783701514920132808624967186918478256380597326874848236541340633025533775381290771516010426828373589306733278474020343552321881584978897001280016142472036103241769952994254247671725270231252614120204754921190373881204524553247891093330171810077513889848254299102628011636459


In [295]:
N = p * q

Using 17 as our encryption exponent, generate d

In [296]:
e = 17
d = RSA_generate_D(e, p, q)

In [297]:
print(d)

8111683511656862534253074603845930193344637059812316703314581723307419155877217321149051386105334428239961401790155762231464907559014784850706859229278069660664175881257347391666524345407049777446518444384741874559663391670232533539760757347666327225855678507598391213034030024837003408358506561793258924920280197533524048710297541086527256176499807540980308251408928069426156965176333262518218422533236706188719144926078632384386741263338938715331390569003816959400230892408989628463706142988819917740967757468316618319059982675748698074075230291808330068360145824112121937955293783423297766446519541122089962323873


Lets say Bob wants to send a message, but he wants to make sure that others can be sure he was the one that truly sent the message

In [298]:
message = "This message is from Bob!"

ascii_message = convert_to_ascii(message)
print(ascii_message)

84104105115032109101115115097103101032105115032102114111109032066111098033


Bob can encrypt the message using his private key d

In [299]:
encrypted_message = RSA_encrypt(ascii_message, d, N)

In [300]:
print(encrypted_message)

5461603548938846822024599455259948669029575061738619524414593515115999090577735329305446540696531404061657008915945528613131973000835098787240497554844654359634645475782413150895680161696017892115680179267315360363870494369757908700329275554044478176698583460383051668530189397140458583720956482895012094511024901880940931578698321213183178338789040105831411696543983669851922741557432641956825968331103640742413762723076196327531075415595957471400898457481083450057092128857089015083567257259523191456544980233827341790823361375465908776581965589849819097763387518531812616898942828139034455528607799518475734649204


In [301]:
print(convert_to_string(encrypted_message))

ǍɛȤΪ̶͎ɗǇăδʝȿ=ˢɫȌƞɑȃsϧZɁ˟ŉıƾȜʸȓƔ=ʑΓαȐɥύ ̓b̓ðǱȪ͌ʎŧɺʅǛ̎ƝͿʨ¡ʸͼsʨ³ċĻŨūͦǮű˵ΌʼŉēȪ,Ǟ°ʺɇǌſ3ʜȒ½ƍǊɇːμǢͿ^ǿ΅ͰάΣɂʺŁÕ·²Œ̕(i̿ƛʸȟϗʝ͓Κ˥ȭưʁμ̹ψŋgʀ˦Ɲ˺˓LÄŇȓKƟɓνǗƐ΂ǉǡSǂ9\͙YSȷāăȋ¿ǈȠϔé̻ŕ̷̖ũŷǑΌ̈Ʌυɍ̳͑a˻ƃȆȓ̬ɨ΂ή̼"ǇȐɟ̟ȆǛ˞ʉÌ


Now anyone that wants to verify if the message was really from Bob can use his public key to decrypt the message

In [302]:
decrypted_message = RSA_decrypt(encrypted_message, e, N)
print(convert_to_string(decrypted_message))

This message is from Bob!


# Blinding Attack

An attacker can execute what is known as a Blinding attack which will allow the attacker to obtain a message encrypted with the victim's private key

In [315]:
bad_message = "Transfer $1000000 to Eve"
bad_ascii_message = convert_to_ascii(bad_message)
print(bad_ascii_message)

84114097110115102101114032036049048048048048048048032116111032069118101


Use a random number and raise to the power of public key and multiply it with the message

In [316]:
random_number = pow(7, e)
print(random_number)
bad_ascii_message = bad_ascii_message * random_number

232630513987207


In [317]:
print(bad_ascii_message)

19567505644295919156614934743979004764790938642435431729180738596448018929073686133907


In [321]:
print(convert_to_string(bad_ascii_message))

ȷǹʄħΗɦΦ˧ϓ˼̖ΪʂƳƯ˙´ˢɔǀΡIʮ΋


If the attacker somehow manages to get the recipient to sign it, the attacker will be able to obtained a message signed with the victim's private key

In [318]:
bad_encrypted_message = RSA_encrypt(bad_ascii_message, d, N)
print(bad_encrypted_message)

6312036335314889907034229202834320248793345720608898608224841681082226394410151679986235724042065109715570758091158229300712341836832731368478870731969980715909633109825363760122891341540691963577863639491464356919712113558048331278855188911762081159413281676409957654689077599522152250801354421905910298741958685694300105916924267872517997566715333301283776139639328503438683777061285244336465129149918434162421209470172371920392807295184597339662074945612781453357530285853371536239468518925340277955710032227599650836637293318018128594401682508991476791528965523311584299581513246656624802853255985351377660467667


Attacker can retrieve the signed message by multipling the inverse of 7 with the encrypted message and then mod N

In [319]:
signed_message = (bad_encrypted_message * pow(7, -1, N)) % N
print(signed_message)

4841680039278317503356383265130068986595015960567253628499202791475350217770384367413287205257171737961634217739669688698241861076783285694411741730216488223166833015157192127398439159417808743842289478628512390060366806462405563616291680556261084818188941228892069682714968526281137691317182390286141520495926739519883342531894027776609136013077221086172936201452533972420647109056051974318917969255970388037333616557599171017407765718420722817487089615284134556492442617987079076701107256121183061525969762534529697805788354591029103325283191203757828654283171831173345246296484076804596074608537746254588078714244


Decrypt with public key, this message is encrypted with Bob's public key!

In [320]:
decrypted_message = RSA_decrypt(signed_message, e, N)
print(convert_to_string(decrypted_message))

Transfer $1000000 to Eve


# Conclusion

The signer can pad the message after receiving it but before signing it

Let f be some sort of padding function,

m'= m * r^e mod N to be the tampered message

The signer pads the m', m'' = f(m') and then signs it, (m'')^d mod N

If the attacker tries to unblind, they will not be able to get the signed message